# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.tile_sources as gts

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodiak,57.7900,-152.4072,278.88,93,100,3.09,US,1700932651
1,1,grytviken,-54.2811,-36.5092,275.82,67,58,3.03,GS,1700932651
2,2,hasaki,35.7333,140.8333,281.37,64,95,4.42,JP,1700932651
3,3,invercargill,-46.4000,168.3500,283.72,92,38,1.79,NZ,1700932651
4,4,bodo,67.2800,14.4050,267.17,77,20,6.17,NO,1700932380


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
gdf = gv.Dataset(city_data_df, kdims=['Lng', 'Lat'], vdims=['City', 'Humidity'])

tiles = gts.CartoDark()

cities = gdf.to(gv.Points, kdims=['Lng', 'Lat'], vdims=['City', 'Humidity']).opts(
    color='City', size=gv.dim('Humidity') * 0.1, cmap='Category20', tools=['hover'], width=1200, height=800,
    colorbar=True, xlabel='x', ylabel='y'
)
map_plot = tiles * cities

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[
    (city_data_df['Max Temp'] > 290) &
    (city_data_df['Cloudiness'] >= 30) &
    (city_data_df['Cloudiness'] >= 60) 
    
]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,adamstown,-25.0660,-130.1015,295.04,82,68,9.82,PN,1700932651
16,16,afaahiti,-17.7500,-149.2833,300.08,78,99,9.59,PF,1700932652
17,17,bredasdorp,-34.5322,20.0403,291.39,60,100,2.92,ZA,1700932652
23,23,nova vicosa,-17.8919,-39.3719,298.22,67,100,7.32,BR,1700932653
26,26,puerto ayora,-0.7393,-90.3518,303.13,88,77,4.02,EC,1700932654


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lng', 'Lat', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lng,Lat,Humidity,Hotel Name
8,adamstown,PN,-130.1015,-25.0660,82,
16,afaahiti,PF,-149.2833,-17.7500,78,
17,bredasdorp,ZA,20.0403,-34.5322,60,
23,nova vicosa,BR,-39.3719,-17.8919,67,
26,puerto ayora,EC,-90.3518,-0.7393,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    'radius': radius,
    'type': 'accommodation.hotel',
    'limit': 1,
    'apiKey': geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lon = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
bredasdorp - nearest hotel: Victoria Hotel
nova vicosa - nearest hotel: Pousada Pontal da Barra
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
alice springs - nearest hotel: Aurora Alice Springs
livingstonia - nearest hotel: Lukwe Lodge
avarua - nearest hotel: Paradise Inn
weno - nearest hotel: High Tide Hotel
puerto baquerizo moreno - nearest hotel: Dory's House
galesong - nearest hotel: No hotel found
lawas - nearest hotel: World Trade Hotel Batangas
plettenberg bay - nearest hotel: Milkwood Manor
atafu village - nearest hotel: No hotel found
chui - nearest hotel: Alerces
camana - nearest hotel: Caja Arequipa
warah - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
wanning - nearest hotel: No hotel found
sorong - nearest hotel: Marina Mamberamo
la troncal - nearest hotel: New York
west island - nearest hotel: Cocos Village Bungalows
boende 

,City,Country,Lng,Lat,Humidity,Hotel Name
8,adamstown,PN,-130.1015,-25.0660,82,No hotel found
16,afaahiti,PF,-149.2833,-17.7500,78,Omati Lodge
17,bredasdorp,ZA,20.0403,-34.5322,60,Victoria Hotel
23,nova vicosa,BR,-39.3719,-17.8919,67,Pousada Pontal da Barra
26,puerto ayora,EC,-90.3518,-0.7393,88,Hostal La Mirada De Solitario George
...,...,...,...,...,...,...
530,pong nam ron,TH,102.2623,12.9058,75,Iyara Resort
533,placido de castro,BR,-67.1856,-10.3353,72,No hotel found
536,titao,BF,-2.0667,13.7667,11,Auberge
537,cabedelo,BR,-34.8339,-6.9811,59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
gdf = gv.Dataset(hotel_df, kdims=['Lng', 'Lat'], vdims=['City', 'Humidity', 'Hotel Name', 'Country'])

tiles = gts.CartoDark()

cities = gdf.to(gv.Points, kdims=['Lng', 'Lat'], vdims=['City', 'Humidity', 'Hotel Name', 'Country']).opts(
    color='City', size=gv.dim('Humidity') * 0.1, cmap='Category20', tools=['hover'], width=1200, height=800,
    colorbar=True, xlabel='x', ylabel='y'
)
map_plot = tiles * cities

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)